<a href="https://www.kaggle.com/code/parthabera/predict-fifa-2018-man-14-08-22-15?scriptVersionId=103251323" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fifa-2018-match-statistics/FIFA 2018 Statistics.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from sklearn.neural_network import MLPClassifier
import tensorflow as tf

In [3]:
data=pd.read_csv("../input/fifa-2018-match-statistics/FIFA 2018 Statistics.csv")

In [4]:
data

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,11-07-2018,England,Croatia,1,46,11,1,6,4,4,...,1,0,0,No,5.0,Semi- Finals,No,0,NaN,NaN
124,14-07-2018,Belgium,England,2,43,12,4,3,5,4,...,1,0,0,Yes,4.0,3rd Place,No,0,NaN,NaN
125,14-07-2018,England,Belgium,0,57,15,5,7,3,5,...,2,0,0,No,NaN,3rd Place,No,0,NaN,NaN
126,15-07-2018,France,Croatia,4,39,8,6,1,1,2,...,2,0,0,Yes,18.0,Final,No,0,1.0,18.0


In [5]:
#preprocessing:
data.drop('Date',axis=1,inplace=True)

In [6]:
data.isnull().sum()

Team                        0
Opponent                    0
Goal Scored                 0
Ball Possession %           0
Attempts                    0
On-Target                   0
Off-Target                  0
Blocked                     0
Corners                     0
Offsides                    0
Free Kicks                  0
Saves                       0
Pass Accuracy %             0
Passes                      0
Distance Covered (Kms)      0
Fouls Committed             0
Yellow Card                 0
Yellow & Red                0
Red                         0
Man of the Match            0
1st Goal                   34
Round                       0
PSO                         0
Goals in PSO                0
Own goals                 116
Own goal Time             116
dtype: int64

In [7]:
data.drop(['Own goals','Own goal Time'],axis=1,inplace=True)

In [8]:
##mean imputation:
data['1st Goal']=data['1st Goal'].fillna(data['1st Goal'].mean())

In [9]:
###encoding:
data.dtypes

Team                       object
Opponent                   object
Goal Scored                 int64
Ball Possession %           int64
Attempts                    int64
On-Target                   int64
Off-Target                  int64
Blocked                     int64
Corners                     int64
Offsides                    int64
Free Kicks                  int64
Saves                       int64
Pass Accuracy %             int64
Passes                      int64
Distance Covered (Kms)      int64
Fouls Committed             int64
Yellow Card                 int64
Yellow & Red                int64
Red                         int64
Man of the Match           object
1st Goal                  float64
Round                      object
PSO                        object
Goals in PSO                int64
dtype: object

In [10]:
data['Team'].unique()

array(['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay', 'Morocco', 'Iran',
       'Portugal', 'Spain', 'France', 'Australia', 'Argentina', 'Iceland',
       'Peru', 'Denmark', 'Croatia', 'Nigeria', 'Costa Rica', 'Serbia',
       'Germany', 'Mexico', 'Brazil', 'Switzerland', 'Sweden',
       'Korea Republic', 'Belgium', 'Panama', 'Tunisia', 'England',
       'Colombia', 'Japan', 'Poland', 'Senegal'], dtype=object)

In [11]:
print(f"unique elments:{data['Team'].unique()} \n")
print(f"unique elments:{data['Opponent'].unique()}\n")
print(f"unique elments:{data['Man of the Match'].unique()}\n")
print(f"unique elments:{data['PSO'].unique()}\n")
print(f"unique elments:{data['Round'].unique()}\n")

unique elments:['Russia' 'Saudi Arabia' 'Egypt' 'Uruguay' 'Morocco' 'Iran' 'Portugal'
 'Spain' 'France' 'Australia' 'Argentina' 'Iceland' 'Peru' 'Denmark'
 'Croatia' 'Nigeria' 'Costa Rica' 'Serbia' 'Germany' 'Mexico' 'Brazil'
 'Switzerland' 'Sweden' 'Korea Republic' 'Belgium' 'Panama' 'Tunisia'
 'England' 'Colombia' 'Japan' 'Poland' 'Senegal'] 

unique elments:['Saudi Arabia' 'Russia' 'Uruguay' 'Egypt' 'Iran' 'Morocco' 'Spain'
 'Portugal' 'Australia' 'France' 'Iceland' 'Argentina' 'Denmark' 'Peru'
 'Nigeria' 'Croatia' 'Serbia' 'Costa Rica' 'Mexico' 'Germany'
 'Switzerland' 'Brazil' 'Korea Republic' 'Sweden' 'Panama' 'Belgium'
 'England' 'Tunisia' 'Japan' 'Colombia' 'Senegal' 'Poland']

unique elments:['Yes' 'No']

unique elments:['No' 'Yes']

unique elments:['Group Stage' 'Round of 16' 'Quarter Finals' 'Semi- Finals' '3rd Place'
 'Final']



In [12]:
label_encoder=LabelEncoder()

data['Man of the Match']=label_encoder.fit_transform(data['Man of the Match'])
man_mappings={index:label for index,label in enumerate(label_encoder.classes_)}

data["PSO"]=label_encoder.fit_transform(data['PSO'])
pso_mappings={index:label for index,label in enumerate(label_encoder.classes_)}

In [13]:
pso_mappings

{0: 'No', 1: 'Yes'}

In [14]:
round_values=list(data['Round'].unique())
round_values

['Group Stage',
 'Round of 16',
 'Quarter Finals',
 'Semi- Finals',
 '3rd Place',
 'Final']

In [15]:
round_mappings={label:index for index,label in enumerate(round_values)}

In [16]:
round_mappings

{'Group Stage': 0,
 'Round of 16': 1,
 'Quarter Finals': 2,
 'Semi- Finals': 3,
 '3rd Place': 4,
 'Final': 5}

In [17]:
data['Round']=data['Round'].apply(lambda x : round_mappings[x] )

In [18]:
###ONE hot encodings
pd.get_dummies(data["Team"])

,Argentina,Australia,Belgium,Brazil,Colombia,Costa Rica,Croatia,Denmark,Egypt,England,...,Portugal,Russia,Saudi Arabia,Senegal,Serbia,Spain,Sweden,Switzerland,Tunisia,Uruguay
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
124,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
data['Opponent']=(data['Opponent'].apply(lambda x : "opp_"+ x))

In [20]:
data_conact=pd.concat([data,pd.get_dummies(data['Team']),pd.get_dummies(data['Opponent'])],axis=1)

In [21]:
data_conact.columns

Index(['Team', 'Opponent', 'Goal Scored', 'Ball Possession %', 'Attempts',
       'On-Target', 'Off-Target', 'Blocked', 'Corners', 'Offsides',
       'Free Kicks', 'Saves', 'Pass Accuracy %', 'Passes',
       'Distance Covered (Kms)', 'Fouls Committed', 'Yellow Card',
       'Yellow & Red', 'Red', 'Man of the Match', '1st Goal', 'Round', 'PSO',
       'Goals in PSO', 'Argentina', 'Australia', 'Belgium', 'Brazil',
       'Colombia', 'Costa Rica', 'Croatia', 'Denmark', 'Egypt', 'England',
       'France', 'Germany', 'Iceland', 'Iran', 'Japan', 'Korea Republic',
       'Mexico', 'Morocco', 'Nigeria', 'Panama', 'Peru', 'Poland', 'Portugal',
       'Russia', 'Saudi Arabia', 'Senegal', 'Serbia', 'Spain', 'Sweden',
       'Switzerland', 'Tunisia', 'Uruguay', 'opp_Argentina', 'opp_Australia',
       'opp_Belgium', 'opp_Brazil', 'opp_Colombia', 'opp_Costa Rica',
       'opp_Croatia', 'opp_Denmark', 'opp_Egypt', 'opp_England', 'opp_France',
       'opp_Germany', 'opp_Iceland', 'opp_Iran', 'opp_J

In [22]:
data_conact.drop(["Team","Opponent"],axis=1,inplace=True)

In [23]:
np.sum(data_conact.dtypes==object)

0

In [24]:
y=data_conact["Man of the Match"]
X=data_conact.drop('Man of the Match',axis=1)

In [25]:
####SCALING:
scaler=RobustScaler()

X = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)

In [26]:
###Splitting the data:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8)

In [27]:
##train:
sk_model=MLPClassifier(hidden_layer_sizes=(16,16))
sk_model.fit(X_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(hidden_layer_sizes=(16, 16))

In [28]:
inputs=tf.keras.Input(shape=(85,))
X=tf.keras.layers.Dense(32,activation=tf.nn.relu)(inputs)
X=tf.keras.layers.Dense(32,activation=tf.nn.relu)(X)
outputs=tf.keras.layers.Dense(2,activation=tf.nn.softmax)(X)

tf_model=tf.keras.Model(inputs=inputs,outputs=outputs)

2022-08-14 06:38:10.288446: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [29]:
tf_model.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=['accuracy'])

In [30]:
tf_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,epochs=200)

2022-08-14 06:38:10.543224: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
3/3 [==============================] - 1s 128ms/step - loss: 0.7121 - accuracy: 0.5309 - val_loss: 0.7476 - val_accuracy: 0.2857
Epoch 2/200
3/3 [==============================] - 0s 26ms/step - loss: 0.6891 - accuracy: 0.5309 - val_loss: 0.7504 - val_accuracy: 0.2857
Epoch 3/200
3/3 [==============================] - 0s 21ms/step - loss: 0.6731 - accuracy: 0.5802 - val_loss: 0.7535 - val_accuracy: 0.3810
Epoch 4/200
3/3 [==============================] - 0s 27ms/step - loss: 0.6583 - accuracy: 0.6420 - val_loss: 0.7565 - val_accuracy: 0.3333
Epoch 5/200
3/3 [==============================] - 0s 27ms/step - loss: 0.6457 - accuracy: 0.6790 - val_loss: 0.7596 - val_accuracy: 0.3333
Epoch 6/200
3/3 [==============================] - 0s 27ms/step - loss: 0.6334 - accuracy: 0.6667 - val_loss: 0.7611 - val_accuracy: 0.3333
Epoch 7/200
3/3 [==============================] - 0s 20ms/step - loss: 0.6217 - accuracy: 0.6914 - val_loss: 0.7625 - val_accuracy: 0.3333
Epoch 8/200
3/3 [==

In [31]:
##results:
sk_score=sk_model.score(X_test,y_test)
tf_score=tf_model.evaluate(X_test,y_test,verbose=False)

In [32]:
print(f"sk_score is {sk_score}")
print(f"tf_score is {tf_score[1]}")

sk_score is 0.6538461538461539
tf_score is 0.5384615659713745
